In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
from datasets import load_dataset
from datasets import get_dataset_split_names
xsum_dataset = load_dataset("xsum")

Found cached dataset xsum (/home/vv2116/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)
100%|██████████| 3/3 [00:00<00:00, 489.84it/s]


In [4]:
xsum_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [5]:
# from transformers import 
from transformers import PegasusForConditionalGeneration,PegasusTokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

base_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

KeyboardInterrupt: 

In [ ]:
from dataset import *
num_samples=204045
val_dataset = Dataset("val", tokenizer,xsum_dataset["train"]["document"][0:] , xsum_dataset["train"]["summary"][0:])


In [ ]:
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 10, shuffle = False)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
model = base_model.to(device)



In [ ]:
from tqdm import tqdm
import gc
import pandas as pd
import pickle



def beam_search_step(batch, tokenizer, base_model, device):
    # 1 - beam search
    if True:
        summary_ids = base_model.generate(
            batch["text_inputs"]['input_ids'],
            attention_mask = batch["text_inputs"]["attention_mask"],
            num_beams = 5,
            num_return_sequences = 5,
            max_length = 64
        )

    generated = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    del summary_ids
    gc.collect()

    return generated


def get_summaries(tokenizer, val_loader, base_model, device):
    val_texts = []
    val_summaries = []
    val_labels = []

    for idx, batch in tqdm(enumerate(val_loader)):
        for k in batch["text_inputs"].keys():
            batch["text_inputs"][k] = batch["text_inputs"][k].to(device)
            if len(batch["text_inputs"][k].shape) > 2:
                batch["text_inputs"][k] = batch["text_inputs"][k].squeeze(1)

        model.zero_grad()
        val_texts += batch["text"]

        raw_summaries = beam_search_step(batch, tokenizer, base_model, device)
        
        summaries = []
        for i in range(len(batch["text"])):
            summaries.append("|".join(raw_summaries[i*5:(i+1)*5]))

        val_summaries += summaries

        labels = batch["summary"]
        val_labels += labels
        df = pd.DataFrame({"text": batch["text"],"summaries":summaries,"labels":labels})
        df.to_csv("candidates_1.csv",mode='a',index=False,header=False)

        # with open(f"candidates_txt", 'a+b') as fp:
        #     pickle.dump(val_texts,fp)
        # with open(f"candidates_summary", 'a+b') as fp:
        #     pickle.dump(val_summaries,fp)
        # with open(f"candidates_label", 'a+b') as fp:
        #     pickle.dump(val_labels,fp)

    # print(len(val_texts), len(val_summaries), len(val_summaries[0]), len(val_labels))

    # return val_texts, val_summaries, val_labels

In [ ]:
get_summaries(tokenizer, val_loader, model, device)


165it [02:51,  1.03it/s]

In [ ]:


# base_results = [val_summaries[i][0] for i in range(len(val_summaries))]
# print("*"*100)
# print("\nTop beam:")

# r1, r2, rl = rouge_eval("true labels", val_texts, base_results, val_labels)

NameError: name 'val_summaries' is not defined

In [ ]:
# # pip install torchmetrics
# from torchmetrics.text.rouge import ROUGEScore
# from nltk.tokenize import sent_tokenize
# import numpy as np


# def pre_rouge_processing(summary):

#     summary = summary.replace("<n>", " ")    
#     summary = "\n".join(sent_tokenize(summary))
    
#     return summary
# def rouge_eval(mode, val_texts, val_summaries, val_labels):
#     print("\n", "*"*10, "1 - ROUGE evaluation with {}".format(mode), "*"*10)
#     scorer = ROUGEScore()
#     all_r1s = []
#     all_r2s = []
#     all_rls = []
#     for i in range(len(val_summaries)):
#         summary = val_summaries[i]
#         summary = pre_rouge_processing(summary)
#         label = val_labels[i]
#         rscores = scorer(summary, label)
#         all_r1s.append(rscores["rouge1_fmeasure"])
#         all_r2s.append(rscores["rouge2_fmeasure"])
#         all_rls.append(rscores["rougeL_fmeasure"])
#     all_r1s = 100 * np.array(all_r1s)
#     all_r2s = 100 * np.array(all_r2s)
#     all_rls = 100 * np.array(all_rls)
#     mean_r1 = np.mean(all_r1s)
#     mean_r2 = np.mean(all_r2s)
#     mean_rl = np.mean(all_rls)
#     mean_r = (mean_r1 + mean_r2 + mean_rl) / 3
#     print("Mean R: {:.4f}, R-1: {:.4f} (var: {:.4f}), R-2: {:.4f} (var: {:.4f}), R-L: {:.4f} (var: {:.4f})".format(
#         mean_r, mean_r1, np.std(all_r1s), mean_r2, np.std(all_r2s), mean_rl, np.std(all_rls)))

#     return all_r1s, all_r2s, all_rls

In [ ]:
# import pandas as pd
# df = pd.read_csv("candidates_1.csv",header=None)
# df.head()

,0,1,2
0,"The full cost of damage in Newton Stewart, one...",The clean-up operation is continuing in parts ...,Clean-up operations are continuing across the ...
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed in a sus...,Two tourist buses have been destroyed by fire ...
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton beat Mercedes team-mate Nico Ro...,Lewis Hamilton stormed to pole position at the...
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer has gone ...,A former Lincolnshire Police officer carried o...
4,Patients and staff were evacuated from Cerahpa...,Turkish police have ended a siege at a psychia...,An armed man who locked himself into a room at...


('The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - bu